In [1]:
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.keys import Keys
import time
import requests

In [18]:
#크롬 웹드라이버 불러오기
driver = webdriver.Chrome()
res = driver.get('https://m.place.naver.com/place/list?query=제주도관광지&level=top')
driver.implicitly_wait(20)


In [19]:
#2차 크롤링을 위한 bs4 셋팅
session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[ 500, 502, 503, 504 ])

session.mount('http://', HTTPAdapter(max_retries=retries))


In [26]:
#body부분을 잡기 위해 쓸데없이 버튼을 클릭해줌
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a').click()
driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[1]/a').click()

#검색결과가 모두 보이지 않기 때문에 page down을 눌러 끝까지 펼쳐준다.
for scroll in range(0,30):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)


In [32]:
from bs4 import BeautifulSoup
html = driver.page_source
bs = BeautifulSoup(html, 'html.parser')
soup = bs.select_one('div.YluNG')
naver_info = soup.select('li.VLTHu')



In [41]:
#2차 크롤링을 위한 url
url = 'https://m.place.naver.com'

for info in naver_info:

    tourist_destination = info.select_one('span.YwYLL').text
    sectors = info.select_one('span.YzBgS').text
    link = info.select_one('div.ouxiq').select_one('a').attrs['href']
    time.sleep(0.06)

    #네이버 플레이스로 이동(place ID로 접속)
    N_res = session.get(link, headers=headers)
    N_soup_srch = BeautifulSoup(N_res.content, 'html.parser')
    mart_oldtel = N_soup_srch.select_one('span.yxkiA > a').attrs['href']
    mart_tel = str(re.sub('tel:', '', mart_oldtel)) # 'tel:' 삭제

    #주소는 '공유'에서 파싱
    address = N_soup_srch.select('span.yxkiA > a')[3].attrs['data-line-description']

    print(tourist_destination, '/',sectors, '/', link, '/',  mart_tel, '/', address )
    time.sleep(0.06)


루나폴 / 테마파크 / https://tivan.naver.com/d/ylsGCXW1sWCq6ZyaDnE-EnwrsMaTeS4uWNw0ppd84nGhBpz1DD7dt5LZqJqaKJajSYVcmZ4tQFYr0iaPgbqiEDwTyRo2RZr8AozQN2VgIcanp83PEccTJWLyYB_Fmna9eCgle3Fu6M_C_r678erYA5hnOAhS7uLmnlCq-qoGJke8hZt8P_u2iqlQeqEQkQb0ASQK5t2XwlTkQGCCxpkt2_ndVDl5HEjG2mlu4OgYp-bw10i1dJyM9F9r8R7oZaaGPr6gwv1S0lHF73S3tepD3-niBWJhbAI4NK10UwFvoRn57_feGt4G427lA8UX0VP97_bxielNEFyQ_aWzrGdWic-M-5xUWirZf8Ir4nU7USAjQdHjxBq5oAt-qv77QCtWh-qvGckjG0ypcMcwNswLc4_h5hvRumnU8ArbpTwVQEouPweNFnAADPZpHcvfsEvgdkLPnKbjplCVVqbSP32TDJYeLP2FqVuUKSwZyifnhZKYmai_znFlgow4_Niab8-xsyxTWeAnMTQ0p3zRQVz4sYH208AWD7erkoCn_5uyg5pEddkk4fRdCfdTt0FUwN_7T3xLU77l_6-FDPihusvJp7Zw9QYSrmOpAQ5SFJkXoxs33p5qJlkW1bH1u58lgzYBlo14eNKSCqt-VbzAyWjDvZ9iJ4c3V8xDNO5oxQioL-AycoGdi0_rQ08FwJDOnxN0tAwdKGqcNf7FunNPsDm1lwhvkNL7t6cRgONucR1N4rMdBp-THJsjPFtW4UsnaDC12yjQZyn0OjkHjchrpk_Gv9_skFfMp1MT7c9oUwPJipMcYsI-escVI0EP6C8eLwEOi3nPf8m1ZLsnmgYqvesblw==?fu=https%3A%2F%2Fm.place.naver.com%2Fplace%2F1823167512%3Fentry%3Dple%26from%3DPLACE_AD%26n_ad_group_typ

In [44]:
import pandas as pd

# 2차 크롤링을 위한 기본 URL
url = 'https://m.place.naver.com'

# 데이터를 담을 빈 리스트 생성
data = {'Tourist Destination': [],
        'Sectors': [],
        'Link': [],
        'Mart Tel': [],
        'Address': []}

for info in naver_info:
    # 각 정보 수집
    tourist_destination = info.select_one('span.YwYLL').text
    sectors = info.select_one('span.YzBgS').text
    link = info.select_one('div.ouxiq').select_one('a').attrs['href']
    time.sleep(0.06)

    # 네이버 플레이스로 이동(place ID로 접속)
    N_res = session.get(link, headers=headers)
    N_soup_srch = BeautifulSoup(N_res.content, 'html.parser')
    mart_oldtel = N_soup_srch.select_one('span.yxkiA > a').attrs['href']
    mart_tel = str(re.sub('tel:', '', mart_oldtel))  # 'tel:' 삭제
    
    # 주소는 '공유'에서 파싱
    address = N_soup_srch.select('span.yxkiA > a')[3].attrs['data-line-description']

    # 데이터를 딕셔너리에 추가
    data['Tourist Destination'].append(tourist_destination)
    data['Sectors'].append(sectors)
    data['Link'].append(link)
    data['Mart Tel'].append(mart_tel)
    data['Address'].append(address)

    time.sleep(0.06)

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# 데이터프레임 출력
print(df)


    Tourist Destination    Sectors  \
0                   루나폴       테마파크   
1                   달제주  관광농원,팜스테이   
2              헬로키티아일랜드       테마파크   
3                동백포레스트    식물원,수목원   
4                 카멜리아힐    식물원,수목원   
..                  ...        ...   
199           동쪽송당 동화마을       자연공원   
200            제주올레길1코스       도보코스   
201              용연구름다리       지역명소   
202                안덕계곡         계곡   
203              걸매생태공원    자연,생태공원   

                                                  Link        Mart Tel  \
0    https://tivan.naver.com/d/ylsGCXW1sWCq6ZyaDnE-...    064-794-9680   
1    https://tivan.naver.com/d/50XzIaQZVk3BlxR_Mq2s...  0507-1300-6921   
2    https://m.place.naver.com/place/33703960?entry...    064-792-6114   
3    https://m.place.naver.com/place/1611903913?ent...  0507-1331-2102   
4    https://m.place.naver.com/place/11846866?entry...    064-800-6296   
..                                                 ...             ...   
199  https://m.place.nave

In [45]:
df

,Tourist Destination,Sectors,Link,Mart Tel,Address
0,루나폴,테마파크,https://tivan.naver.com/d/ylsGCXW1sWCq6ZyaDnE-...,064-794-9680,제주 서귀포시 안덕면 일주서로 1836 루나폴
1,달제주,"관광농원,팜스테이",https://tivan.naver.com/d/50XzIaQZVk3BlxR_Mq2s...,0507-1300-6921,제주 서귀포시 표선면 원님로652번길 86 달제주 세컨농장
2,헬로키티아일랜드,테마파크,https://m.place.naver.com/place/33703960?entry...,064-792-6114,제주 서귀포시 안덕면 한창로 340
3,동백포레스트,"식물원,수목원",https://m.place.naver.com/place/1611903913?ent...,0507-1331-2102,제주 서귀포시 남원읍 생기악로 53-38
4,카멜리아힐,"식물원,수목원",https://m.place.naver.com/place/11846866?entry...,064-800-6296,제주 서귀포시 안덕면 병악로 166
...,...,...,...,...,...
199,동쪽송당 동화마을,자연공원,https://m.place.naver.com/place/1130170517?ent...,064-743-5000,제주 제주시 구좌읍 비자림로 1191 동쪽송당 동화마을
200,제주올레길1코스,도보코스,https://m.place.naver.com/place/19932808?entry...,064-762-2190,제주 서귀포시 성산읍 오조리 396-1
201,용연구름다리,지역명소,https://m.place.naver.com/place/17099516?entry...,#,제주 제주시 용담이동 2581
202,안덕계곡,계곡,https://m.place.naver.com/place/11491209?entry...,064-794-9001,제주 서귀포시 안덕면 감산리 1946
